# 인증서 체인

## ✅ 보기:

> “CA 간에는 인증 체인을 형성할 수 있기 때문에, 특정 CA에 의해 서명된 인증서는 인증 체인상의 다른 CA에 의해서도 보장될 수 있다.”
>
>
> → ✔️ 맞는 보기
>

---

## 🔐 왜 맞는 말인가요? (실무 + 이론)

### 🔗 1. 인증서 체인(Certificate Chain)이란?

> 인증서가 직접 루트 CA에 의해 서명되지 않고,
>
>
> 중간 CA를 거쳐서 신뢰가 이어지는 구조
>

```
[루트 CA] (자체서명)
   ↓ 서명
[중간 CA]
   ↓ 서명
[서버 인증서] (예: www.example.com)

```

---

### 📌 체인 구조의 핵심

- **루트 CA는 브라우저/OS에 미리 신뢰된 상태로 탑재**
- 브라우저는:
    - 서버 인증서 → 중간 CA 인증서 → 루트 CA 인증서
    - 이 전체 **체인을 따라 올라가면서 검증**

→ 따라서 중간 CA가 발급한 인증서라도,

**루트 CA가 신뢰되면 체인 상의 인증서도 “간접적으로 신뢰”됩니다**

---

### ✅ 보기에서의 표현 해석

- “특정 CA에 의해 서명된 인증서”

    → 예: 중간 CA가 서명한 인증서

- “인증 체인상의 다른 CA에 의해서도 보장될 수 있다”

    → 루트 CA가 **중간 CA를 신뢰하면**, 그 아래 인증서들도 **결국 신뢰받는다**


→ 즉, **하위 CA의 인증서도 상위 CA(루트)의 권한 아래 신뢰된다는 의미**로 맞는 설명입니다.

---

## 🧠 시험용 정리

| 개념 | 요점 |
| --- | --- |
| 인증 체인 | 여러 CA들이 서명 관계로 연결된 구조 |
| 루트 CA | 최상위 CA, 스스로 서명(Self-signed) |
| 중간 CA | 루트 CA가 서명한 하위 CA |
| 서버 인증서 | 중간 CA가 서명한 최종 엔티티 인증서 |
| 신뢰 검증 흐름 | 서버 인증서 → 중간 CA → 루트 CA (거슬러 올라가며 신뢰 확인) |

---

## ❌ 오답 보기 예시 (참고용)

> “인증서는 반드시 루트 CA에 의해 직접 서명되어야 한다.”
>
>
> → ❌ 틀린 보기 (중간 CA를 통한 체인이 일반적임)
>
>

---

Python에서는 **`cryptography` 라이브러리**를 사용해서 **X.509 인증서 체인을 검증**할 수 있습니다.

아래에 간단한 실습 코드를 단계별로 보여드릴게요.

---

## ✅ 실습 개요

### 📁 전제: 세 개의 인증서 파일이 준비되어 있어야 합니다.

| 파일명 | 내용 |
| --- | --- |
| `leaf.pem` | 서버(최종 사용자) 인증서 |
| `intermediate.pem` | 중간 CA 인증서 |
| `root.pem` | 루트 CA 인증서 (신뢰 앵커) |

> Let’s Encrypt 기준이면 fullchain.pem 안에서 추출 가능
>

---

## 🐍 1. Python 인증서 체인 검증 예제

In [ ]:
from cryptography import x509
from cryptography.hazmat.backends import default_backend

# 인증서 로드 함수
def load_cert(path):
    with open(path, 'rb') as f:
        return x509.load_pem_x509_certificate(f.read(), default_backend())

# 인증서 로드
leaf_cert = load_cert("leaf.pem")
intermediate_cert = load_cert("intermediate.pem")
root_cert = load_cert("root.pem")

# 1. issuer가 일치하는지 확인
print("Leaf issued by:", leaf_cert.issuer)
print("Intermediate subject:", intermediate_cert.subject)
print("→ 일치 여부:", leaf_cert.issuer == intermediate_cert.subject)

print("Intermediate issued by:", intermediate_cert.issuer)
print("Root subject:", root_cert.subject)
print("→ 일치 여부:", intermediate_cert.issuer == root_cert.subject)

## ✅ 예상 출력 결과

```
Leaf issued by: <Name(C=US,O=Let's Encrypt,CN=R3)>
Intermediate subject: <Name(C=US,O=Let's Encrypt,CN=R3)>
→ 일치 여부: True

Intermediate issued by: <Name(C=US,O=Internet Security Research Group,CN=ISRG Root X1)>
Root subject: <Name(C=US,O=Internet Security Research Group,CN=ISRG Root X1)>
→ 일치 여부: True

```

---

## 📌 심화: 체인 검증 (전체 신뢰성 검증은 수동 또는 OpenSSL 연동 필요)

Python 표준 `cryptography`는 체인 자체를 자동으로 검증하지 않기 때문에,

- 직접 서명 확인을 하거나,
- OpenSSL의 `verify` 명령과 연동하는 방법도 사용합니다.

---

## 🔧 OpenSSL을 활용한 체인 검증 예 (외부에서)

```bash
openssl verify -CAfile root.pem -untrusted intermediate.pem leaf.pem

```

- `CAfile`: 루트 인증서
- `untrusted`: 중간 인증서 체인
- `leaf.pem`: 최종 엔티티 인증서

---

## 🧠 공무원 시험용으로 요약

- 인증서 체인은 **루트 → 중간 → 사용자 인증서**로 구성
- Python에서 **issuer/subject 비교**로 체인 구조 확인 가능
- 실질적인 검증은 OpenSSL `verify`가 더 정확

---

## ✅ 인증서 체인 구조 실습 (공시용 개념 확인용 예시)

In [3]:

# 가짜 인증서 구조를 딕셔너리로 흉내내는 예제
class FakeCert:
    def __init__(self, subject, issuer, name):
        self.subject = subject
        self.issuer = issuer
        self.name = name

# 인증서 구성: 루트 → 중간 → 서버
root = FakeCert(subject="RootCA", issuer="RootCA", name="루트 인증서")
intermediate = FakeCert(subject="IntermediateCA", issuer="RootCA", name="중간 인증서")
leaf = FakeCert(subject="www.example.com", issuer="IntermediateCA", name="서버 인증서")

# 체인 추적 함수
def verify_chain(cert, parents):
    print(f"[{cert.name}] 주체: {cert.subject}, 발급자: {cert.issuer}")

    # 루트 CA 여부 확인 (자체서명)
    if cert.subject == cert.issuer:
        print(f"→ 🏁 {cert.name}은 자체 서명된 루트 CA입니다. 체인 종료")
        return

    for parent in parents:
        if cert.issuer == parent.subject:
            print(f"→ 🔗 {cert.issuer}는 신뢰되는 상위 CA입니다: {parent.name}")
            return verify_chain(parent, parents)

    print(f"❌ {cert.name}의 발급자 {cert.issuer}를 찾을 수 없습니다. (체인 실패)")


# 체인 검증 실행
verify_chain(leaf, [intermediate, root])


[서버 인증서] 주체: www.example.com, 발급자: IntermediateCA
→ 🔗 IntermediateCA는 신뢰되는 상위 CA입니다: 중간 인증서
[중간 인증서] 주체: IntermediateCA, 발급자: RootCA
→ 🔗 RootCA는 신뢰되는 상위 CA입니다: 루트 인증서
[루트 인증서] 주체: RootCA, 발급자: RootCA
→ 🏁 루트 인증서은 자체 서명된 루트 CA입니다. 체인 종료


## 📌 이 코드로 얻는 학습 효과

| 개념 | 확인 |
| --- | --- |
| 인증 체인은 “주체(subject) → 발급자(issuer)”로 이어짐 | ✔️ |
| 루트 CA는 자기 자신을 서명 | ✔️ |
| 중간 CA가 서버 인증서를 서명하면 루트 CA까지 따라 올라가야 신뢰 | ✔️ |

---

## 🧠 시험 대비 정리 포인트

- 인증 체인은 **루트 → 중간 → 서버** 구조
- 루트 CA는 **자체서명(Self-signed)**
- 인증서는 **발급자(issuer) 필드**를 기준으로 검증 경로를 형성